In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movies.head(2)
# deleting the unuseful cols
# budget
# homepage
# id
# original language
# original_title
# popularity
# production_companies
# production_countries
# release_date
# revenue
# runtime
# spoken_languages
# status
# tagline
# vote_average
# vote_count

In [ ]:
credits.head()

In [ ]:
movies.shape

In [ ]:
# merging two datasets movies & credits
movies=movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
movies['original_language'].value_counts()

In [ ]:
# final useful cols in movies dataframe
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
# if there is any duplicate values in dataset
movies.duplicated().sum()

In [ ]:
# if there is any missing values in dataset
movies.isnull().sum()

In [ ]:
# to omit missing values
movies.dropna(inplace=True)

In [ ]:
# to convert string into list
import ast

def convert(text):
    L=[]
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'][0]

In [ ]:
movies['genres']= movies['genres'].apply(convert)
movies.head(2)

In [ ]:
movies['keywords']= movies['keywords'].apply(convert)
movies.head(2)

In [ ]:
# preprocessing on cast col
import ast
ast.literal_eval('[{"id": 28, "name": "Action"},{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

def convert3(text):
    L=[]
    counter=0
    for i in ast.literal_eval(text):
        if counter<3:
            L.append(i['name'])
        counter+=1
    return L

In [ ]:
movies['cast']= movies['cast'].apply(convert3)
movies.head(2)

In [ ]:
# preprocessing on crew col

def fetch_director(text):
    L=[]
    for i in ast.literal_eval(text):
        if i['job']=='Director':
            L.append(i['name'])
    return L

In [ ]:
movies['crew']= movies['crew'].apply(fetch_director)
movies.head()

In [ ]:
# convert string into list
movies['overview']=movies['overview'].apply(lambda x:x.split())
movies.head()

In [ ]:
# to remove the space in between two words in cols 'cast','crew','genres','keywords'

def collapse(L):
    L1=[]
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
collapse(['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver'])

In [ ]:
movies['cast']=movies['cast'].apply(collapse)
movies['crew']=movies['crew'].apply(collapse)
movies['genres']=movies['genres'].apply(collapse)
movies['keywords']=movies['keywords'].apply(collapse)

In [ ]:
movies.head()

In [ ]:
# to concatenate all cols into a single col 'tags'
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
# to delete all cols except 'tags'
new= movies.drop(columns=['overview','genres','keywords','cast','crew'])
new.head()

In [ ]:
# to convert the list into strings
new['tags']=new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:
new['tags'][0]

In [ ]:
x=new['tags'].values.tolist()
y=''
for i in x:
    y=y+i
    
len(y)

In [ ]:
len(set(y.split()))

In [ ]:
# taking top 5000 words which are frequently occurs
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
# convert the dataframe into array of vectors
cv.fit_transform(new['tags']).toarray()

In [ ]:
vector=cv.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape
# this vector is a sparse matrix where max values are 0

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
cosine_similarity(vector).shape

In [ ]:
# similarity of 1st movie with all other movies including itself
similarity[0]

In [ ]:
# index calculate of desired movie
new[new['title']=='The Dark Knight Rises'].index[0]

In [ ]:
def recommend(movie):
    index=new[new['title']==movie].index[0]
    distances=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x:x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [ ]:
movies.sample(5)

In [ ]:
recommend('Sense and Sensibility')

In [ ]:
recommend('The Gambler')

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Titanic')

In [ ]:
recommend('Gandhi')

In [ ]:
import pickle

In [ ]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))